# Extraindo Entidades Nomeadas

In [ ]:
import pandas as pd
import numpy as np

# Lendo arquivo de entrada

In [ ]:
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:10000] #apenas para processar mais rápido
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Exsitem 457 sentenças contendo 2.746 palavras diferentes e 17 tags.

In [ ]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(457, 2746, 17)

In [ ]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,28
1,B-eve,10
2,B-geo,244
3,B-gpe,303
4,B-nat,5
5,B-org,176
6,B-per,160
7,B-tim,149
8,I-art,20
9,I-eve,10


# Transformando dados para vetor e criando treino/teste -> para algoritmos tradicionais!

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values

classes = np.unique(y)
classes = classes.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((6700, 3242), (6700,))

In [ ]:
X

# CRF

In [ ]:
!pip install sklearn_crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

     |████████████████████████████████| 747kB 4.3MB/s 


Função para recuperar sentenças com os POS e as tags

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

# Criando o formato de entrada do CRF (extração de características)

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]
    
def sent2tokens(sent):
    return [token for token, postag, label in sent]

# Divisão de treinamento e teste

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
X

In [ ]:
y

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O'],
 ['O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'B-org', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O'],
 ['O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

# Treinando o modelo CRF

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

# Avaliação

In [ ]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-art       0.50      0.40      0.44         5
       B-eve       0.00      0.00      0.00         2
       B-geo       0.79      0.68      0.73        77
       B-gpe       0.75      0.88      0.81        91
       B-nat       0.00      0.00      0.00         2
       B-org       0.77      0.68      0.72        53
       B-per       0.85      0.92      0.88        61
       B-tim       0.95      0.89      0.92        45
       I-art       0.00      0.00      0.00         4
       I-eve       0.00      0.00      0.00         1
       I-geo       0.75      0.38      0.50        16
       I-gpe       0.67      0.57      0.62         7
       I-nat       0.00      0.00      0.00         2
       I-org       0.74      0.70      0.72        50
       I-per       0.87      0.97      0.92        75
       I-tim       0.33      1.00      0.50         1
           O       0.99      1.00      1.00      2884

    accuracy              

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Entendendo o funcionamento do algoritmo

In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top-20 transições mais prováveis:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("Top-20 transições menos prováveis:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top-20 transições mais prováveis:
B-org  -> I-org   4.680201
B-per  -> I-per   4.242035
I-org  -> I-org   4.198692
I-per  -> I-per   3.991878
B-art  -> I-art   3.688363
B-eve  -> I-eve   3.636531
I-art  -> I-art   3.464745
B-gpe  -> I-gpe   3.365085
O      -> O       3.119727
B-geo  -> I-geo   2.966060
B-tim  -> I-tim   2.372497
I-tim  -> I-tim   2.276223
I-eve  -> I-eve   1.833767
O      -> B-tim   1.764286
I-geo  -> I-geo   1.733762
O      -> B-gpe   1.572374
O      -> B-geo   1.331913
B-gpe  -> B-per   1.262858
O      -> B-art   1.110531
O      -> B-org   1.102532
Top-20 transições menos prováveis:
B-gpe  -> I-art   -0.389697
I-org  -> B-geo   -0.457163
B-geo  -> I-gpe   -0.470893
B-gpe  -> I-org   -0.489994
B-eve  -> O       -0.505690
O      -> I-gpe   -0.537264
O      -> I-eve   -0.579971
O      -> I-geo   -0.604511
B-geo  -> I-org   -0.612946
B-gpe  -> B-gpe   -0.624199
B-tim  -> B-gpe   -0.627473
B-geo  -> I-art   -0.651209
B-org  -> I-per   -0.664598
B-geo  -> I-per   -0.781181

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top-20 positivo:")
print_state_features(Counter(crf.state_features_).most_common(20))

print("Top-20 negativo:")
print_state_features(Counter(crf.state_features_).most_common()[-20:])

Top-20 positivo:
5.860922 O        bias
3.722280 O        BOS
3.721897 B-tim    word[-3:]:day
3.690997 B-tim    word[-2:]:ay
3.149401 B-gpe    word.istitle()
3.047903 B-tim    word[-2:]:0s
2.978401 B-geo    -1:word.lower():in
2.502835 O        postag:NN
2.437516 B-tim    -1:word.lower():in
2.386781 B-gpe    postag:JJ
2.223646 B-org    word.isupper()
2.059408 B-gpe    word[-2:]:na
2.041420 B-org    word[-3:]:ban
2.033487 B-gpe    postag[:2]:JJ
1.980111 B-tim    word[-3:]:ber
1.962547 O        word[-2:]:ic
1.914448 B-tim    word.isdigit()
1.906829 I-tim    word.isdigit()
1.823769 B-gpe    -1:word.lower():with
1.813563 O        +1:word.lower():men
Top-20 negativo:
-1.070367 B-gpe    word[-3:]:can
-1.106395 O        word[-2:]:sh
-1.129731 I-tim    word.istitle()
-1.141161 B-gpe    -1:word.istitle()
-1.156629 O        word.isupper()
-1.177472 O        word[-3:]:ish
-1.192073 O        postag:NNPS
-1.203677 B-org    -1:postag:NNP
-1.217778 B-art    -1:word.istitle()
-1.394852 O        word[-2

In [ ]:
!pip install eli5
import eli5
eli5.show_weights(crf, top=10)

     |████████████████████████████████| 112kB 4.4MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.12,1.111,-1.29,0.688,-0.58,1.332,-0.605,1.572,-0.537,0.0,1.103,-1.871,0.028,-0.97,1.764,-1.494
B-art,0.0,0.0,3.688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011,0.0,-0.216,0.0,0.0
I-art,-0.058,0.0,3.465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.229,0.0,0.0
B-eve,-0.506,0.0,0.0,0.0,3.637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.074,0.0,0.0
I-eve,0.0,0.0,0.0,0.0,1.834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-geo,0.197,0.0,-0.651,0.0,-0.284,-0.182,2.966,0.687,-0.471,0.0,-0.17,-0.613,-0.224,-0.781,1.057,-0.217
I-geo,-0.246,0.0,0.0,0.0,0.0,0.0,1.734,0.0,0.0,0.0,0.0,0.0,0.0,-0.041,0.0,0.0
B-gpe,0.435,0.0,-0.39,0.0,-0.14,-0.206,-0.244,-0.624,3.365,0.0,0.785,-0.49,1.263,-0.828,-0.271,0.0
I-gpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934,0.0,0.0,0.0,0.0,-0.093,0.0,0.0
B-nat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
